# VacationPy
----


In [83]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [84]:
city_data = pd.read_csv("../output_data/clean_city_data.csv")

# Visualize
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mataura,-46.19,168.86,32.09,92,40,3.71,NZ,1602785419
1,mahebourg,-20.41,57.70,75.20,78,40,8.05,MU,1602785420
2,salym,60.06,71.48,31.05,96,0,3.89,RU,1602785420
3,kodiak,57.79,-152.41,41.00,80,1,5.82,US,1602785420
4,hermanus,-34.42,19.23,59.00,84,43,1.01,ZA,1602785420


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [85]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
humidity = city_data["Humidity"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure(center=(10,20), zoom_level=2, \
                   layout={'width': '800px', 'margin': '0 auto 0 auto'})


heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3, opacity=0.5)


fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [86]:
ideal_city = city_data.loc[(city_data['Max Temp'] > 70) & 
                           (city_data['Max Temp'] < 80) &
                           (city_data['Wind Speed'] < 10) &
                           (city_data['Cloudiness'] == 0)
                          ].reset_index(drop=True)
ideal_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kununurra,-15.77,128.73,77.00,88,0,2.24,AU,1602785432
1,jhargram,22.45,86.98,77.36,89,0,2.84,IN,1602785386
2,marzuq,14.40,46.47,73.90,19,0,4.12,YE,1602785476
3,tura,25.52,90.22,73.67,94,0,1.83,IN,1602785476
4,kanye,-24.97,25.33,74.25,32,0,7.76,BW,1602785487
5,port hedland,-20.32,118.57,75.20,69,0,9.17,AU,1602785463
6,alangayam,12.60,78.75,71.33,83,0,2.24,IN,1602785500
7,alice springs,-23.70,133.88,75.00,45,0,4.99,AU,1602785450


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
hotel_df = pd.DataFrame(columns=['Hotel Name'])
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_city.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request and print url
    hotel_results = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:   
        hotel_df.loc[index, "Hotel Name"] = hotel_results["results"][0]["name"]
        hotel_df.loc[index, "Lat"] = hotel_results["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = hotel_results["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "Address"] = hotel_results["results"][0]["vicinity"]
        print(f"Retieved information of Hotel {hotel_results['results'][0]['name']}.")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


hotel_df

Retieved information of Hotel Kununurra Lakeside Resort.
Retieved information of Hotel Jhargram Prokriti Porjoton Kendra.
Missing field/result... skipping.
Retieved information of Hotel Hotel Polo Orchid.
Retieved information of Hotel Ga-Thuli Guest House.
Retieved information of Hotel The Esplanade Hotel.
Missing field/result... skipping.
Retieved information of Hotel DoubleTree by Hilton Hotel Alice Springs.


,Hotel Name,Lat,Lng,Address
0,Kununurra Lakeside Resort,-15.783161,128.742868,"50 Casuarina Way, Kununurra"
1,Jhargram Prokriti Porjoton Kendra,22.424326,87.002846,Jhargram
3,Hotel Polo Orchid,25.514468,90.192206,"Araimile, Tura"
4,Ga-Thuli Guest House,-24.977493,25.327671,"Kanye, Mmalekwa Ward, Kanye"
5,The Esplanade Hotel,-20.313925,118.576081,"2 Anderson St, Port Hedland"
7,DoubleTree by Hilton Hotel Alice Springs,-23.715016,133.879153,"82 Barrett Dr, Alice Springs"


In [88]:
hotel_df[["City","Country"]] = ideal_city[["City","Country"]]
hotel_df

,Hotel Name,Lat,Lng,Address,City,Country
0,Kununurra Lakeside Resort,-15.783161,128.742868,"50 Casuarina Way, Kununurra",kununurra,AU
1,Jhargram Prokriti Porjoton Kendra,22.424326,87.002846,Jhargram,jhargram,IN
3,Hotel Polo Orchid,25.514468,90.192206,"Araimile, Tura",tura,IN
4,Ga-Thuli Guest House,-24.977493,25.327671,"Kanye, Mmalekwa Ward, Kanye",kanye,BW
5,The Esplanade Hotel,-20.313925,118.576081,"2 Anderson St, Port Hedland",port hedland,AU
7,DoubleTree by Hilton Hotel Alice Springs,-23.715016,133.879153,"82 Barrett Dr, Alice Springs",alice springs,AU


In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [90]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(10,20), zoom_level=2, \
                   layout={'width': '800px', 'margin': '0 auto 0 auto'})

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='800px'))